In [1]:
import sqlite3
import uuid
from datetime import datetime, timedelta
import random

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('ai_cost_tracking.db')
cursor = conn.cursor()

In [3]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('ai_cost_tracking.db')
cursor = conn.cursor()

In [4]:
# Create tables
cursor.executescript('''
-- Request Keys table
CREATE TABLE IF NOT EXISTS request_keys (
    request_key_id INTEGER PRIMARY KEY AUTOINCREMENT,
    key_name TEXT NOT NULL,
    key_value TEXT NOT NULL UNIQUE,
    is_active INTEGER DEFAULT 1,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Model Information table
CREATE TABLE IF NOT EXISTS model_information (
    model_id INTEGER PRIMARY KEY AUTOINCREMENT,
    model_name TEXT NOT NULL,
    model_input_price REAL NOT NULL,
    model_output_price REAL NOT NULL,
    price_effective_date DATE NOT NULL,
    is_current INTEGER DEFAULT 1
);

-- API Versions table
CREATE TABLE IF NOT EXISTS api_versions (
    api_version_id INTEGER PRIMARY KEY AUTOINCREMENT,
    api_version TEXT NOT NULL,
    release_date DATE NOT NULL
);

-- Token Tracking table
CREATE TABLE IF NOT EXISTS token_tracking (
    tracking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    request_id TEXT NOT NULL,
    request_key_id INTEGER,
    input_token_count INTEGER NOT NULL,
    output_token_count INTEGER NOT NULL,
    model_id INTEGER,
    api_version_id INTEGER,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (request_key_id) REFERENCES request_keys(request_key_id),
    FOREIGN KEY (model_id) REFERENCES model_information(model_id),
    FOREIGN KEY (api_version_id) REFERENCES api_versions(api_version_id)
);
''')

In [7]:
# Simulate token tracking data
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
current_date = start_date

while current_date <= end_date:
    for _ in range(random.randint(50, 200)):  # Random number of requests per day
        request_id = str(uuid.uuid4())
        request_key_id = random.randint(1, 2)
        input_token_count = random.randint(100, 1000)
        output_token_count = random.randint(50, 500)
        model_id = random.randint(1, 2)
        api_version_id = random.randint(1, 2)
        timestamp = current_date + timedelta(seconds=random.randint(0, 86399))

        cursor.execute('''
        INSERT INTO token_tracking
        (request_id, request_key_id, input_token_count, output_token_count, model_id, api_version_id, timestamp)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (request_id, request_key_id, input_token_count, output_token_count, model_id, api_version_id, timestamp))

    current_date += timedelta(days=1)

conn.commit()

In [8]:
# Example queries
top_5_requests = '''
SELECT tt.request_id, tt.input_token_count, tt.output_token_count, 
       mi.model_name, tt.timestamp,
       (tt.input_token_count * mi.model_input_price / 1000.0 + 
        tt.output_token_count * mi.model_output_price / 1000.0) as total_cost
FROM token_tracking tt
JOIN model_information mi ON tt.model_id = mi.model_id
ORDER BY total_cost DESC
LIMIT 5
'''

total_cost_per_model = '''
SELECT mi.model_name, 
       SUM(tt.input_token_count * mi.model_input_price / 1000.0 + 
           tt.output_token_count * mi.model_output_price / 1000.0) as total_cost
FROM token_tracking tt
JOIN model_information mi ON tt.model_id = mi.model_id
GROUP BY mi.model_name
'''

total_cost_by_key = '''
SELECT 
    rk.key_name,
    SUM(tt.input_token_count * mi.model_input_price / 1000.0 + 
        tt.output_token_count * mi.model_output_price / 1000.0) as total_cost
FROM token_tracking tt
JOIN request_keys rk ON tt.request_key_id = rk.request_key_id
JOIN model_information mi ON tt.model_id = mi.model_id
GROUP BY rk.key_name
ORDER BY total_cost DESC;'''

monthly_cost_by_key = '''
SELECT 
    rk.key_name,
    strftime('%Y-%m', tt.timestamp) as month,
    SUM(tt.input_token_count * mi.model_input_price / 1000.0 + 
        tt.output_token_count * mi.model_output_price / 1000.0) as monthly_cost
FROM token_tracking tt
JOIN request_keys rk ON tt.request_key_id = rk.request_key_id
JOIN model_information mi ON tt.model_id = mi.model_id
GROUP BY rk.key_name, month
ORDER BY rk.key_name, month;'''

monthly_cost_for_all_keys = '''
SELECT 
    strftime('%Y-%m', tt.timestamp) as month,
    SUM(tt.input_token_count * mi.model_input_price / 1000.0 + 
        tt.output_token_count * mi.model_output_price / 1000.0) as total_monthly_cost
FROM token_tracking tt
JOIN model_information mi ON tt.model_id = mi.model_id
GROUP BY month
ORDER BY month;'''

montly_usage_per_model_per_key = '''
SELECT 
    rk.key_name,
    mi.model_name,
    strftime('%Y-%m', tt.timestamp) as month,
    SUM(tt.input_token_count * mi.model_input_price / 1000.0 + 
        tt.output_token_count * mi.model_output_price / 1000.0) as monthly_cost
FROM token_tracking tt
JOIN request_keys rk ON tt.request_key_id = rk.request_key_id
JOIN model_information mi ON tt.model_id = mi.model_id
GROUP BY rk.key_name, mi.model_name, month
ORDER BY rk.key_name, mi.model_name, month;'''

In [9]:
queries = [
    ("Top 5 Most Expensive Invocations", top_5_requests),
    ("Total Cost Per Model", total_cost_per_model),
    ("Total cost by API key", total_cost_by_key),
    ("Monthly cost by API key", monthly_cost_by_key),
    ("Total monthly cost across all keys", monthly_cost_for_all_keys),
    ("Monthly cost breakdown by model for each API key", montly_usage_per_model_per_key)
]

for title, query in queries:
    print(f"\n{title}:")
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        print(row)

# Close the connection
conn.close()


Top 5 Most Expensive Invocations:
('4986360e-c187-4fbe-a50f-9f434bc9ee33', 1000, 500, 'GPT-4', '2023-04-16 18:25:18', 0.06)
('fe899c9e-e678-4a4c-bb47-a8d916badf97', 999, 500, 'GPT-4', '2023-11-04 18:52:23', 0.059969999999999996)
('27c7c6cb-2a17-4e69-90d3-c16c091a70b1', 999, 498, 'GPT-4', '2023-12-09 04:07:10', 0.05985)
('8bae4453-e2bb-468f-8d49-d157e30291f1', 999, 497, 'GPT-4', '2023-05-30 20:13:50', 0.059789999999999996)
('ae7b852c-0e93-43ac-b16a-9e9052ceb80b', 994, 499, 'GPT-4', '2023-04-20 04:24:29', 0.059759999999999994)

Total Cost Per Model:
('GPT-3.5-Turbo', 94.93769800000072)
('GPT-4', 2280.037110000019)

Total cost by API key:
('API Key 2', 1193.122457499968)
('API Key 1', 1181.852350499967)

Monthly cost by API key:
('API Key 1', '2023-01', 103.4957614999999)
('API Key 1', '2023-02', 93.35396550000009)
('API Key 1', '2023-03', 98.94001350000015)
('API Key 1', '2023-04', 98.03596849999964)
('API Key 1', '2023-05', 98.42651149999962)
('API Key 1', '2023-06', 93.98076849999997)